In [ ]:
!pip install datasets
!pip install transformers
from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import json
import pandas as pd
with open('/content/drive/MyDrive/sync/quran_datasets_qa/qrcd_v1.1_train.jsonl', 'r') as train:
    train = list(train)
with open('/content/drive/MyDrive/sync/quran_datasets_qa/qrcd_v1.1_dev.jsonl', 'r') as dev:
    dev = list(dev)
with open("/content/drive/MyDrive/sync/quran_datasets_qa/AQQAC.json") as extra:
    extra = json.load(extra)

In [ ]:
def json2list(json_list):
  new_data = []
  for json_str in json_list:
      result = json.loads(json_str)
      new_sample={
              'answers':result['answers'],
              'passage':result['passage'],
              'question': result['question'],
              }
      new_data.append(new_sample)
  return new_data

In [ ]:
train_data=json2list(train)
dev_data=json2list(dev)

In [ ]:
def split_multi_answers(data):
    new_data = []
    for i in range(0,len(data)):
        new_sample={
                'passage':data[i]['passage'],
                'question': data[i]['question'],
                'answers':data[i]['answers']
                }
        new_data.append(new_sample)
    return new_data
extra_data=split_multi_answers(extra)

In [ ]:
train_df = pd.DataFrame(train_data)
dev_df = pd.DataFrame(dev_data)
extra_data = pd.DataFrame(extra_data)
train_df.head()

,answers,passage,question
0,"[{'text': 'أولئك الذين اشتروا الضلالة بالهدى',...",ومن الناس من يقول آمنا بالله وباليوم الآخر وما...,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...
1,[{'text': 'أولئك الذين اشتروا الضلالة بالهدى و...,إن الذين يكتمون ما أنزل الله من الكتاب ويشترون...,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...
2,"[{'text': 'ليجزي الله كل نفس ما كسبت', 'start_...",فلا تحسبن الله مخلف وعده رسله إن الله عزيز ذو ...,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...
3,"[{'text': 'كل إنسان ألزمناه طائره في عنقه', 's...",وجعلنا الليل والنهار آيتين فمحونا آية الليل وج...,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...
4,"[{'text': 'من شاء فليؤمن ومن شاء فليكفر', 'sta...",وقل الحق من ربكم فمن شاء فليؤمن ومن شاء فليكفر...,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...


In [ ]:
from transformers import AutoTokenizer
# import the AutoTokenizer which will be used to download the pretrained tokenizer model.
token_transformer_name = "timpal0l/mdeberta-v3-base-squad2"
# Downloading the pretrained tokenizer
tokenizer = AutoTokenizer.from_pretrained(token_transformer_name) # inputs id , attention mask

In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["passage"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer[0]["start_char"]
        end_char = answer[0]["start_char"] + len(answer[0]["text"])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
from datasets import Dataset
train_df = Dataset.from_pandas(train_df)
dev_df = Dataset.from_pandas(dev_df)

In [ ]:
tokenized_train = train_df.map(preprocess_function, batched=True, remove_columns=train_df.column_names)

Map:   0%|          | 0/710 [00:00<?, ? examples/s]

In [ ]:
tokenized_dev = dev_df.map(preprocess_function, batched=True, remove_columns=train_df.column_names)

Map:   0%|          | 0/109 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
model = AutoModelForQuestionAnswering.from_pretrained("timpal0l/mdeberta-v3-base-squad2")

In [ ]:
from transformers import default_data_collator as data_collator
# Configure the TrainingArguments to be available to be passed to Trainer class.
training_args = TrainingArguments(
    output_dir="mdeberta_Quran_qa",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=10,
    num_train_epochs=10,
    weight_decay=0.01,
    push_to_hub=True,
)
# Pass the Trainer parameters.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_dev,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

Cloning https://huggingface.co/hemagamal/mdeberta_Quran_qa into local empty directory.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,2.239507
2,No log,2.328242
3,No log,2.422559
4,No log,2.655124
5,No log,2.933221
6,1.031700,3.212425
7,1.031700,3.291508
8,1.031700,3.540054
9,1.031700,3.613162
10,1.031700,3.624823


Adding files tracked by Git LFS: ['tokenizer.json']. This may take a bit of time if the files are large.


TrainOutput(global_step=890, training_loss=0.6607370526603099, metrics={'train_runtime': 815.5847, 'train_samples_per_second': 8.705, 'train_steps_per_second': 1.091, 'total_flos': 1391430356121600.0, 'train_loss': 0.6607370526603099, 'epoch': 10.0})

In [ ]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/1.04G [00:00<?, ?B/s]

Upload file runs/May12_17-07-25_9993c72eefa2/events.out.tfevents.1683911250.9993c72eefa2.19044.0:   0%|       …

To https://huggingface.co/hemagamal/mdeberta_Quran_qa
   63687e9..5996847  main -> main

   63687e9..5996847  main -> main

To https://huggingface.co/hemagamal/mdeberta_Quran_qa
   5996847..aae5b82  main -> main

   5996847..aae5b82  main -> main



'https://huggingface.co/hemagamal/mdeberta_Quran_qa/commit/5996847b885f68323623f43f42079c8a4350332e'

In [ ]:
from scipy.special import softmax
import torch
import numpy as np
def predict_quran_qa(passage , question, show_all=False):
    #Inilize the list of answers sorted by its rank
    ranked_answers=[]

    # Pass the question and the passage to the tokenizer
    inputs = tokenizer(question, passage, add_special_tokens=True, return_tensors="pt").to("cuda")

    # Obtain the input_ids from inputs
    input_ids = inputs["input_ids"].tolist()[0]

    # predict the inputs from the qa_model
    outputs = model(**inputs)
    '''
    # The prediction output has two lists start_logits 'for the answer star index'
    # and the end_logits for the answer end index.
    '''
    #obtain the starts and ends scores prediced from the qa_model
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    # Obtain the top 5 start scores
    # convert the 5 start scores into distribution.
    answer_starts_probs = softmax(torch.topk(answer_start_scores , 3).values.cpu().data.numpy())


    # obtain the top 5 indices for the start character
    answer_starts =  torch.topk(answer_start_scores , 3).indices

    # Obtain the top 5 end scores
    # convert the 5 end scores into distribution.
    answer_ends_probs = softmax(torch.topk(answer_end_scores, 3).values.cpu().data.numpy())

    # obtain the full probability by multiplying the matrix elementwise
    full_probs = softmax((np.multiply(answer_starts_probs,answer_ends_probs)))[0] #check[1]

    # obtain the top 5 indices for the end character
    answer_ends = torch.topk(answer_end_scores, 3).indices +1

    print(f"Question: {question}")
    print('top predicted answers:')
    idx =0

    # loop on each answer_start and answer_end indicies
    #This loop mainly will be used to convert the indcies to the words according to the indcies obtained from above.
    for answer_start ,  answer_end in zip(answer_starts.tolist()[0], answer_ends.tolist()[0]):

        idx+=1
        # use the convert_tokens_to_string API to convert the input_ids
        #'from the answer_start to the answer_end' back to the words starting from
        answer = tokenizer.convert_tokens_to_string( tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))


        #If the answer is not empty
        if answer.strip() !='':

            # if the full_probs is good, then register the answer
            if (full_probs[idx-1] > 0.1):

                # print the answer
                print(f"Answer number {idx}: {answer}")
                #Append the answer to the ranked_answers
                ranked_answers.append( { 'answer': answer, 'rank' : len(ranked_answers)+1, 'score':float(full_probs[idx-1])})
    #This is just for checking the ranked_answers if empty.
    if len(ranked_answers) == 0:
        print(' Empty Answer List')
    return ranked_answers

In [ ]:
text= 'سبحان الذي أسرى بعبده ليلا من المسجد الحرام إلى المسجد الأقصى الذي باركنا حوله لنريه من آياتنا إنه هو السميع البصير . واتينا موسى الكتاب وجعلناه هدى لبني إسرائيل أن لا تتخذوا من دوني وكيلا .'
q='لماذا آسرى الله برسوله صلى الله عليه وسلم ؟'
predict_quran_qa(text ,q, show_all=True)

Question: لماذا آسرى الله برسوله صلى الله عليه وسلم ؟
top predicted answers:
Answer number 1: سبحان الذي أسرى بعبده ليلا من المسجد الحرام إلى المسجد الأقصى الذي باركنا حوله لنريه من آياتنا
Answer number 2: الذي أسرى بعبده ليلا


[{'answer': 'سبحان الذي أسرى بعبده ليلا من المسجد الحرام إلى المسجد الأقصى الذي باركنا حوله لنريه من آياتنا',
  'rank': 1,
  'score': 0.5676991939544678},
 {'answer': 'الذي أسرى بعبده ليلا', 'rank': 2, 'score': 0.21617047488689423}]

In [ ]:
with open('/content/drive/MyDrive/sync/quran_datasets_qa/qrcd_v1.1_test_gold.jsonl', 'r') as dev:
    test = list(dev)

In [ ]:
test=json2list(test)

In [ ]:
test[3]

{'answers': [{'text': 'روح القدس', 'start_char': 102}],
 'passage': 'تلك الرسل فضلنا بعضهم على بعض منهم من كلم الله ورفع بعضهم درجات وآتينا عيسى ابن مريم البينات وأيدناه بروح القدس ولو شاء الله ما اقتتل الذين من بعدهم من بعد ما جاءتهم البينات ولكن اختلفوا فمنهم من آمن ومنهم من كفر ولو شاء الله ما اقتتلوا ولكن الله يفعل ما يريد. يا أيها الذين آمنوا أنفقوا مما رزقناكم من قبل أن يأتي يوم لا بيع فيه ولا خلة ولا شفاعة والكافرون هم الظالمون.',
 'question': 'من هم الملائكة المذكورون في القرآن؟'}

In [ ]:
a=3
predict_quran_qa(test[a]['passage'] ,test[a]['question'], show_all=True)

Question: من هم الملائكة المذكورون في القرآن؟
top predicted answers:
Answer number 1: تلك الرسل فضلنا بعضهم على بعض منهم من كلم الله ورفع بعضهم درجات وآتينا عيسى ابن مريم البينات وأيدناه بروح القدس


[{'answer': 'تلك الرسل فضلنا بعضهم على بعض منهم من كلم الله ورفع بعضهم درجات وآتينا عيسى ابن مريم البينات وأيدناه بروح القدس',
  'rank': 1,
  'score': 0.4883165955543518}]